In [31]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from bs4 import BeautifulSoup
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif


In [32]:
test = pd.read_csv('test.csv')
train= pd.read_csv('train.csv')
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [33]:
stops = set(stopwords.words("english"))
english_stemmer=nltk.stem.SnowballStemmer('english')
def cleanData(text,lowercase=False,remove_stops=False,stemming=False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r' ',txt)
    txt = re.sub(r'\n',r' ',txt)
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    b=[]
    stemmer = english_stemmer #PorterStemmer()
    for word in txt:
        b.append(stemmer.stem(word))

    # 5. Return a list of words
    return(b)

In [34]:
clean_train_reviews=[]
for txt in train['text']:
    clean_train_reviews.append("".join(cleanData(txt,True,True,True)))
    
clean_test_reviews=[]
for txt in test['text']:
    clean_test_reviews.append("".join(cleanData(txt,True,True,True)))

In [35]:
vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 200000, ngram_range = ( 1, 4 ),sublinear_tf = True )
vectorizer = vectorizer.fit(clean_train_reviews)
train_features = vectorizer.transform(clean_train_reviews)
test_features = vectorizer.transform(clean_test_reviews)

In [36]:
fselect = SelectKBest(chi2 , k=10000)
train_features = fselect.fit_transform(train_features, train["author"])
test_features = fselect.transform(test_features)

In [37]:
model1 = MultinomialNB(alpha=0.001)
model1.fit( train_features, train["author"] )

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [38]:
pred_1 = model1.predict( test_features.toarray() )

array(['', '', '', ..., '', '', ''], 
      dtype='|S3')